# check ch_func_to_all_pk

The problem with that function is that it mixes things: 
* it does not touch VK. 
* it transforms VP into tuple

In [14]:
    >>> from i2.signatures import ch_func_to_all_pk, Sig
    >>> def f(a, /, b, *, c=None, **kwargs):
    ...     return a + b * c
    ...
    >>> print(Sig(f))
    #(a, /, b, *, c=None, **kwargs)
    >>> ff = ch_func_to_all_pk(f)
    >>> print(Sig(ff))
    #(a, b, c=None, **kwargs)
    >>> ff(1, 2, 3)
    #7
    >>>
    >>> def g(x, y=1, *args, **kwargs):
    ...     ...
    ...
    >>> print(Sig(g))
    #(x, y=1, *args, **kwargs)
    >>> gg = ch_func_to_all_pk(g)
    >>> print(Sig(gg))
    #(x, y=1, args=(), **kwargs)

(a, /, b, *, c=None, **kwargs)
(a, b, c=None, **kwargs)
(x, y=1, *args, **kwargs)
(x, y=1, args=(), **kwargs)


In [18]:
from i2.signatures import VP, VK,PK
from i2.wrapper import nice_kinds
def non_variadics_to_PK(kinds):
    return {name: PK for name,kind in kinds.items() if kind not in [VP, VK]}


kinds_modifier=non_variadics_to_PK
h = nice_kinds(f, kinds_modifier=non_variadics_to_PK)


In [19]:
hh = nice_kinds(g, kinds_modifier=non_variadics_to_PK)
print(Sig(hh))
#(x, y=1, args=(), **kwargs)

(x, y=1, *args, **kwargs)


In [20]:
    >>> def g(x, y=1, *args, **kwargs):
    ...     ...
    ...
    >>> print(Sig(g))
    #(x, y=1, *args, **kwargs)
    >>> gg = ch_func_to_all_pk(g)
    >>> print(Sig(gg))
    #(x, y=1, args=(), **kwargs)

(x, y=1, *args, **kwargs)
(x, y=1, args=(), **kwargs)


In [15]:
def nice_kinds(func, kinds_modifier=convert_to_PK):
    """Wraps the func, changing the argument kinds according to kinds_modifier.
    The default behaviour is to change all kinds to POSITIONAL_OR_KEYWORD kinds.
    The original purpose of this function is to remove argument-kind restriction
    annoyances when doing functional manipulations such as:

    >>> from functools import partial
    >>> isinstance_of_str = partial(isinstance, class_or_tuple=str)
    >>> isinstance_of_str('I am a string')
    Traceback (most recent call last):
      ...
    TypeError: isinstance() takes no keyword arguments

    Here, instead, we can just get a kinder version of the function and do what we
    want to do:

    >>> _isinstance = nice_kinds(isinstance)
    >>> isinstance_of_str = partial(_isinstance, class_or_tuple=str)
    >>> isinstance_of_str('I am a string')
    True
    >>> isinstance_of_str(42)
    False

    See also: ``i2.signatures.all_pk_signature``

    """
    from i2 import Sig, call_somewhat_forgivingly

    sig = Sig(func)
    kinds_modif = kinds_modifier(sig.kinds)

    sig = sig.ch_kinds(**kinds_modif)

    # sig = sig.ch_kinds(**{name: Sig.POSITIONAL_OR_KEYWORD for name in sig.names})

    @wraps(func)
    def _func(*args, **kwargs):
        return call_somewhat_forgivingly(func, args, kwargs, enforce_sig=sig)

    _func.__signature__ = sig
    return _func

# Removing the variadics

Use only ch_variadics_to_non_variadic_kind to do the removal, and tuple_the_args to remove only the VP.

# Work at the signature level signatures

# Wrapper part

In [21]:
# from functools import wraps

# def wrap_from_sig(func, new_sig):
#     @wraps(func)
#     def _func(*args, **kwargs):
#         return call_somewhat_forgivingly(func, args, kwargs, enforce_sig=new_sig)

#     _func.__signature__ = new_sig
    
#     return _func

In [17]:
# new_sig = Sig(ff)
# new_f = wrap_from_sig(f, new_sig)

In [22]:
#Sig(new_f), Sig(f)

# Remove_variadics at the signature level

In [25]:
    >>> from i2.signatures import _remove_variadics_from_sig

    >>> def foo(a, *args, bar, **kwargs):
    ...     return f"{a=}, {args=}, {bar=}, {kwargs=}"
    >>> sig = Sig(foo)
    >>> assert str(sig) == '(a, *args, bar, **kwargs)'
    >>> new_sig = remove_variadics_from_sig(sig)
    >>> str(new_sig)=='(a, args=(), *, bar, kwargs={})'
    

    

    #Note that if there is not variadic positional arguments, the variadic keyword
    #will still be a keyword-only kind.

    >>> def func(a, bar=None, **kwargs):
    ...     return f"{a=}, {bar=}, {kwargs=}"
    >>> nsig = remove_variadics_from_sig(Sig(func))
    >>> assert str(nsig)=='(a, bar=None, *, kwargs={})'
    
    #If the function has neither variadic kinds, it will remain untouched.

    >>> def func(a, /, b, *, c=3):
    ...     return a + b + c
    >>> sig = remove_variadics_from_sig(Sig(func))

    >>> assert sig == Sig(func)
    

    #If you only want the variadic positional to be handled, but leave leave any
    #VARIADIC_KEYWORD kinds (**kwargs) alone, you can do so by setting
    #`ch_variadic_keyword_to_keyword=False`.
    
    >>> def foo(a, *args, bar=None, **kwargs):
    ...     return f"{a=}, {args=}, {bar=}, {kwargs=}"
    >>> assert str(Sig(remove_variadics_from_sig(Sig(foo))))=='(a, args=(), *, bar=None, kwargs={})'
    #<Sig (a, args=(), *, bar=None, **kwargs)>

In [26]:
print(Sig(remove_variadics_from_sig(Sig(foo))))

(a, args=(), *, bar=None, kwargs={})


In [ ]:
from tested import validate_codec
from i2 import wrap

In [27]:
from i2 import wrap

def variadic_dispatch_ingress(x,vp,y, vk):#create an ingress
    return (x,)+ vp, dict(y=y, **vk)

def sum_of_args(x,*vp, y, **vk):
    t = sum(vp)
    return {k: t + v for k, v in vk.items()}

f = wrap(sum_of_args, ingress=variadic_dispatch_ingress)
assert str(Sig(f)) == '(x, vp, y, vk)'
assert sum_of_args(5,1,2,3, y=3, a=4, b=5) == {'a': 10, 'b': 11}

assert f(5,(1,2,3), 3, dict( a=4, b=5)) == {'a': 10, 'b': 11}

from tested import validate_codec
assert validate_codec(f) == True


In [28]:
gg = wrap(func)

In [30]:

validate_codec(gg)

True

# Set debug=True in kwargs_from_args_and_kwargs

In [5]:
from i2.signatures import _remove_variadics_from_sig, ch_variadics_to_non_variadic_kind
from i2.wrapper import InnerMapIngress
from i2 import Sig
from i2.wrapper import wrap
from inspect import Parameter, Signature

In [6]:
    # >>> def foo(a, *args, bar, **kwargs):
    # ...     return f"{a=}, {args=}, {bar=}, {kwargs=}"
    # >>> assert str(Sig(foo)) == '(a, *args, bar, **kwargs)'
    # >>> wfoo = ch_variadics_to_non_variadic_kind(foo)
    # >>> str(Sig(wfoo))
    # '(a, args=(), *, bar, kwargs={})'

    # #And now to do this:

    # >>> foo(1, 2, 3, bar=4, hello="world")
    # #"a=1, args=(2, 3), bar=4, kwargs={'hello': 'world'}"

    # #We can do it like this instead:

    # >>> wfoo(1, (2, 3), bar=4, kwargs=dict(hello="world"))
    # #"a=1, args=(2, 3), bar=4, kwargs={'hello': 'world'}"

In [7]:
#wfoo(1, (2, 3), bar=4, kwargs=dict(hello="world"))

In [18]:
def foo(po, *vp, ko, **vk):
    return f"{po=}, {vp=}, {ko=}, {vk=}"

new_sig = _remove_variadics_from_sig(Sig(foo))
assert str(new_sig)=='(po, vp=(), *, ko, vk={})'

ingress = InnerMapIngress.from_signature(
         foo, outer_sig=new_sig
)
#new_sig.wrap

In [19]:
# from i2.signatures import KO
# ingress = InnerMapIngress.from_signature(
#          foo, outer_sig=new_sig
# )
new_foo = wrap(foo, ingress=ingress)
# assert str(Sig(new_foo))=='(po, vp=(), *, ko, vk={})'
# #ingress2 = InnerMapIngress(foo, kwargs=dict(kind=KO, default={}))

In [20]:
#clean_foo = ch_variadics_to_non_variadic_kind(foo)
#clean_foo(1, (2, 3), ko=4, vk=dict(hello="world"))

In [21]:
#Sig(clean_foo)

In [22]:
Sig(ingress)

<Sig (po, vp=(), *, ko, vk={})>

In [23]:
assert foo(1, 2, 3, ko=4, hello="world")=="po=1, vp=(2, 3), ko=4, vk={'hello': 'world'}"
#new_foo = wrap(foo, ingress=ingress)
assert str(Sig(new_foo))=='(po, vp=(), *, ko, vk={})'
new_foo(1, (2, 3), ko=4, vk=dict(hello="world"))


TypeError: foo() missing 1 required keyword-only argument: 'ko'

In [15]:

# before vk:vk:vk


TypeError: foo() missing 1 required keyword-only argument: 'ko'

In [12]:
def foo(po, *vp, ko, **vk):
    return f"{po=}, {vp=}, {ko=}, {vk=}"
    
ingress_args = (1,(2,3))
ingress_kwargs = {'ko':4, 'vk' :{'hello':"world"}}

inner_sig = Sig(foo)
outer_sig = Sig('(po, vp=(), *, ko, vk={})')

 
func_kwargs = outer_sig.kwargs_from_args_and_kwargs(
            ingress_args, ingress_kwargs, debug=True
        )
print(func_kwargs)
# {'po': 1, 'vp': (2, 3), 'ko': 4, 'vk': {'hello': 'world'}}

#assert (inner_sig.args_and_kwargs_from_kwargs(func_kwargs)
#        ==((1, (2, 3)), {'ko': 4, 'vk': {'vk': {'hello': 'world'}}}))
print(inner_sig.args_and_kwargs_from_kwargs(func_kwargs))
# ((1, (2, 3)), {'ko': 4, 'vk': {'vk': {'hello': 'world'}}})


{'po': 1, 'vp': (2, 3), 'ko': 4, 'vk': {'hello': 'world'}}
((1, (2, 3)), {'ko': 4, 'vk': {'vk': {'hello': 'world'}}})


In [13]:
# def foo(w, /, x: float, y=1, *, z: int = 1, **vk):
#     return ((w + x) * y) ** z
# foo_sig = Sig(foo)
# args, kwargs = foo_sig.args_and_kwargs_from_kwargs(
#              dict(w=4, x=3, y=2, z=1, vk=12)
# )
        

## My changes to kwargs_from... broke args_from_...

In [14]:
def foo(w, /, x: float, y=1, *args, z: int = 1, **rest):
        return ((w + x) * y) ** z

foo_sig = Sig(foo)
args, kwargs = foo_sig.args_and_kwargs_from_kwargs(dict(w=4, x=3, y=2, z=1, t=12))
print(f"args:{args}, kwargs: {kwargs}")

args:(4, 3, 2), kwargs: {'z': 1, 'rest': {'t': 12}}


In [21]:
from i2.signatures import ch_variadics_to_non_variadic_kind
foo_clean = ch_variadics_to_non_variadic_kind(foo)
Sig(foo_clean)

<Sig (w, /, x: float, y=1, args=(), *, z: int = 1, rest={})>

In [22]:
foo_clean(1, (2, 3), bar=4, kwargs ={'hello':"world"})

KeyError: 'rest'

In [49]:
ingress(1, (2, 3), bar=4, kwargs ={'hello':"world"})

TypeError: missing a required argument: 'ko'

In [50]:
def egress_flatten(key):
    def f(output):
        args, kwargs = output
        kwargs[key] = kwargs[key][key]
        return args, kwargs
    return f

def flatten_key(ingress_func, key):
     new_func = wrap(ingress_func, egress = egress_flatten(key))
     return new_func

def mk_ingress(func):
    sig = Sig(func)
    vk_name = sig.var_keyword_name or None
    new_sig = _remove_variadics_from_sig(sig)
    ingress = InnerMapIngress.from_signature(
         func, outer_sig=new_sig
    )
    if vk_name:
         ingress = flatten_key(ingress, vk_name)
    return ingress

    

In [51]:
nfoo = wrap(foo, ingress = mk_ingress(foo))
nfoo(1, (2, 3), bar=4, kwargs ={'hello':"world"})

TypeError: Got unexpected keyword arguments: kwargs, bar

In [52]:
mk_ingress(foo)(1, (2, 3), bar=4, kwargs ={'hello':"world"})

TypeError: Got unexpected keyword arguments: kwargs, bar

In [53]:
ingress(1, (2, 3), bar=4, kwargs ={'hello':"world"})

TypeError: missing a required argument: 'ko'

In [54]:
foo_clean(1, (2, 3), bar=4, kwargs ={'hello':"world"})

KeyError: 'rest'

In [55]:
* find the VK name
* flatten

SyntaxError: invalid syntax (<ipython-input-55-35c4c33c2ddb>, line 1)

In [56]:
foo(1, 2, 3, bar=4, hello="world")

9

In [57]:
validate_codec(new_foo)

False

# Changes made to meshed in the presence of variadic keywords

In [ ]:
from meshed import DAG, FuncNode

def g(x):
    return x

def f(a, **kwargs):
    print(a)
    return a 

f_node = FuncNode(func=f)
d = DAG([f_node])

#d(a=1, x=3) # does not work anymore, as variadic kw are mapped to a dict
d(a=1, kwargs ={'x':3})

In [ ]:
Sig(d)

# Scrap

In [25]:
from i2.signatures import expand_nested_key

In [27]:
k='vk'
v={'vk':{'vk':{'vk':34}}}
v2 = {'vk':{'a':23,'vk':{'vk':34, 'rest':89}}}
v3 = {"ko": 4, "vk": {"hello": "world"}}
expand_nested_key(v3,k)

dict_items([('ko', 4), ('vk', {'hello': 'world'})])

In [5]:
from i2 import Sig
from i2.signatures import _remove_variadics_from_sig, ch_variadics_to_non_variadic_kind
from i2.wrapper import InnerMapIngress
from i2.wrapper import wrap
from inspect import Parameter, Signature

def foo(po, *vp, ko, **vk):
    return f"{po=}, {vp=}, {ko=}, {vk=}"


new_sig = _remove_variadics_from_sig(Sig(foo))
assert str(new_sig) == "(po, vp=(), *, ko, vk={})"

ingress = InnerMapIngress.from_signature(foo, outer_sig=new_sig)


In [12]:
ingress_args = (1,(2,3))
ingress_kwargs = {'ko': 4, 'vk': {'hello': 'world'}}
ingress(*ingress_args, **ingress_kwargs)
#func_kwargs = ingress.outer_sig.kwargs_from_args_and_kwargs(
 #                ingress_args, ingress_kwargs, apply_defaults=True
#             )
#func_kwargs

((1, (2, 3)), {'vk': {'hello': 'world'}})

In [2]:
from i2 import Sig
from i2.signatures import _remove_variadics_from_sig, ch_variadics_to_non_variadic_kind
from i2.wrapper import InnerMapIngress
from i2.wrapper import wrap
from inspect import Parameter, Signature


# def bar(w, /, x: float, y=1, *args, z: int = 1, **rest):
#     return ((w + x) * y) ** z


# bar_sig = Sig(bar)


def foo(po, *vp, ko, **vk):
    return f"{po=}, {vp=}, {ko=}, {vk=}"


new_sig = _remove_variadics_from_sig(Sig(foo))
assert str(new_sig) == "(po, vp=(), *, ko, vk={})"

ingress = InnerMapIngress.from_signature(foo, outer_sig=new_sig)

#if __name__ == "__main__":
new_foo = wrap(foo, ingress=ingress)
ingress_args = (1, (2, 3))
ingress_kwargs = {"ko": 4, "vk": {"hello": "world"}}
print(ingress(*ingress_args, **ingress_kwargs))
    # assert (
    #     foo(1, 2, 3, ko=4, hello="world")
    #     == "po=1, vp=(2, 3), ko=4, vk={'hello': 'world'}"
    # )
    # new_foo = wrap(foo, ingress=ingress)
    # assert str(Sig(new_foo)) == "(po, vp=(), *, ko, vk={})"
    # new_foo(1, (2, 3), ko=4, vk=dict(hello="world"))

    # args, kwargs = foo_sig.args_and_kwargs_from_kwargs(dict(w=4, x=3, y=2, z=1, t=12))

    # print(f"args:{args}, kwargs: {kwargs}")


((1, (2, 3)), {'ko': 4, 'vk': {'vk': {'hello': 'world'}}})


In [5]:
from i2.signatures import expand_nested_key
d = {'ko': 4, 'vk': {'vk': {'hello': 'world'}}}
expand_nested_key(d, k='vk')

dict_items([('ko', 4), ('vk', {'vk': {'hello': 'world'}})])

In [1]:
from i2.wrapper import items_with_mapped_keys
self = ingress
func_kwargs = self.outer_sig.kwargs_from_args_and_kwargs(
    ingress_args, ingress_kwargs, apply_defaults=True
)

# Modify the keys of func_kwargs so they reflect the inner signature's names
# That is, map outer names to inner names.
func_kwargs = dict(
    items_with_mapped_keys(func_kwargs, self.inner_name_for_outer_name)
)
func_kwargs = dict(
    func_kwargs,  # by default, keep the func_kwargs, but
    **self.kwargs_trans(func_kwargs),  # change those that kwargs_trans desires
)
print(f'{func_kwargs=}')
self.inner_sig.args_and_kwargs_from_kwargs(
            func_kwargs, apply_defaults=True, allow_excess=True
        )

NameError: name 'ingress' is not defined

In [18]:
self.inner_sig

<Sig (po, *vp, ko, **vk)>

In [45]:
from typing import Union
def args_and_kwargs_from_kwargs2(
        self,
        kwargs,
        apply_defaults=True,
        allow_partial=False,
        allow_excess=False,
        ignore_kind=False,
        args_limit: Union[int, None] = 0,
    ):
        """Extract args and kwargs such that func(*args, **kwargs) can be called,
        where func has instance's signature.

        :param kwargs: The {argname: argval,...} dict to process
        :param args_limit: How "far" in the params should args (positional arguments)
            be searched for.
            - args_limit==0: Take the minimum number possible of args (positional
                arguments). Only those that are position only or before a var-positional.
            - args_limit is None: Take the maximum number of args (positional arguments).
                The only kwargs (keyword arguments) you should have are keyword-only
                and var-keyword arguments.
            - args_limit positive integer: Take the args_limit first argument names
                (of signature) as args, and the rest as kwargs.

        >>> def foo(w, /, x: float, y=1, *, z: int = 1):
        ...     return ((w + x) * y) ** z
        >>> foo_sig = Sig(foo)
        >>> args, kwargs = foo_sig.args_and_kwargs_from_kwargs(
        ...     dict(w=4, x=3, y=2, z=1)
        ... )
        >>> assert (args, kwargs) == ((4,), {"x": 3, "y": 2, "z": 1})
        >>> assert foo(*args, **kwargs) == foo(4, 3, 2, z=1) == 14

        The `args_limit` begs explanation.
        Consider the signature of `def foo(w, /, x: float, y=1, *, z: int = 1): ...`
        for instance. We could call the function with the following (args, kwargs) pairs:
        - ((1,), {'x': 2, 'y': 3, 'z': 4})
        - ((1, 2), {'y': 3, 'z': 4})
        - ((1, 2, 3), {'z': 4})
        The two other combinations (empty args or empty kwargs) are not valid
        because of the / and * constraints.

        But when asked for an (args, kwargs) pair, which of the three valid options
        should be returned? This is what the `args_limit` argument controls.

        If `args_limit == 0`, the least args (positional arguments) will be returned.
        It's the default.

        >>> kwargs = dict(w=4, x=3, y=2, z=1)
        >>> foo_sig.args_and_kwargs_from_kwargs(kwargs, args_limit=0)
        ((4,), {'x': 3, 'y': 2, 'z': 1})

        If `args_limit is None`, the least kwargs (keyword arguments) will be returned.

        >>> foo_sig.args_and_kwargs_from_kwargs(kwargs, args_limit=None)
        ((4, 3, 2), {'z': 1})

        If `args_limit` is a positive integer, the first `args_limit` arguments
        will be returned (not checking at all if this is valid!).

        >>> foo_sig.args_and_kwargs_from_kwargs(kwargs, args_limit=1)
        ((4,), {'x': 3, 'y': 2, 'z': 1})
        >>> foo_sig.args_and_kwargs_from_kwargs(kwargs, args_limit=2)
        ((4, 3), {'y': 2, 'z': 1})
        >>> foo_sig.args_and_kwargs_from_kwargs(kwargs, args_limit=3)
        ((4, 3, 2), {'z': 1})

        Note that 'args_limit''s behavior is consistent with list behvior in the sense
        that:

        >>> args = (0, 1, 2, 3)
        >>> args[:0]
        ()
        >>> args[:None]
        (0, 1, 2, 3)
        >>> args[2]
        2

        By default, only the arguments that were given in the kwargs input will be
        returned in the (args, kwargs) output.
        If you also want to get those that have defaults (according to signature),
        you need to specify it with the `apply_defaults=True` argument.

        >>> foo_sig.args_and_kwargs_from_kwargs(dict(w=4, x=3))
        ((4,), {'x': 3})
        >>> foo_sig.args_and_kwargs_from_kwargs(dict(w=4, x=3), apply_defaults=True)
        ((4,), {'x': 3, 'y': 1, 'z': 1})

        By default, all required arguments must be given.
        Not doing so will lead to a `TypeError`.
        If you want to process your arguments anyway, specify `allow_partial=True`.

        >>> foo_sig.args_and_kwargs_from_kwargs(dict(w=4))
        Traceback (most recent call last):
          ...
        TypeError: missing a required argument: 'x'
        >>> foo_sig.args_and_kwargs_from_kwargs(dict(w=4), allow_partial=True)
        ((4,), {})

        Specifying argument names that are not recognized by the signature will
        lead to a `TypeError`.
        If you want to avoid this (and just take from the input `kwargs` what ever you
        can), specify this with `allow_excess=True`.

        >>> foo_sig.args_and_kwargs_from_kwargs(dict(w=4, x=3, extra='stuff'))
        Traceback (most recent call last):
            ...
        TypeError: Got unexpected keyword arguments: extra
        >>> foo_sig.args_and_kwargs_from_kwargs(dict(w=4, x=3, extra='stuff'),
        ...     allow_excess=True)
        ((4,), {'x': 3})

        An edge case: When a `VAR_POSITIONAL` follows a `POSITION_OR_KEYWORD`...

        >>> Sig(lambda a, *b, c=2: None).args_and_kwargs_from_kwargs(
        ...     {"a": 1, "b": [2, 3], "c": 4}
        ... )
        ((1, [2, 3]), {'c': 4})

        See `kwargs_from_args_and_kwargs` (namely for the description of the arguments.
        """

        if args_limit is None:
            # Take the maximum number of args (positional arguments).
            # The only kwargs (keyword arguments) you should have are keyword-only
            # and var-keyword arguments.
            idx = next((i for i, p in enumerate(self.params) if p.kind > VP), None)
            names_for_args = self.names[:idx]
        elif args_limit == 0:
            # Take the minimum number possible of args (positional arguments)
            # Only those that are position only or before a var-positional.
            vp_idx = self.index_of_var_positional
            if vp_idx is None:
                names_for_args = self.names_of_kind[PO]
            else:
                # When there's a VP present, all arguments before it can only be
                # expressed positionally if the VP argument is non-empty.
                # So, here we just consider all arguments positionally up to the VP arg.
                names_for_args = self.names[: (vp_idx + 1)]
        else:
            names_for_args = self.names[:args_limit]

        args = tuple(kwargs[name] for name in names_for_args if name in kwargs)
        kwargs = {name: kwargs[name] for name in kwargs if name not in names_for_args}
        print(f'{kwargs=}')
        kwargs = self.kwargs_from_args_and_kwargs(
            args,
            kwargs,
            apply_defaults=apply_defaults,
            allow_partial=allow_partial,
            allow_excess=allow_excess,
            ignore_kind=ignore_kind,
        )
        print(f'{kwargs=}')

        kwargs = {name: kwargs[name] for name in kwargs if name not in names_for_args}
        print(f'{kwargs=}')

        return args, kwargs

In [46]:
args_and_kwargs_from_kwargs2(self.inner_sig,
            func_kwargs, apply_defaults=True, allow_excess=True
        )

args=(1, (2, 3))
kwargs={'ko': 4, 'vk': {'hello': 'world'}}
kwargs={'vk': {'hello': 'world'}}
kwargs={'vk': {'hello': 'world'}}


((1, (2, 3)), {'vk': {'hello': 'world'}})

In [48]:
self.sig

AttributeError: 'InnerMapIngress' object has no attribute 'sig'

In [66]:
def inner_most_kv(k, v):
    if isinstance(v, dict) and k in v:
        return inner_most_kv(k, v[k])
    else:
        return k,v


In [68]:
k='vk'
v={'vk':{'vk':{'vk':34}}}
inner_most_kv(k, v)


('vk', 34)

In [ ]:
def flatten_dict(d: MutableMapping, parent_key: str = '', sep: str ='.') -> MutableMapping:
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, MutableMapping):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [ ]:
counter = 0
string1 = ''

for i in range(1_000_000):
     old_id = id(string1)
     string1 += str(i)
     if id(string1) != old_id:
         counter += 1

print(counter)  # returns 39

In [ ]:
s='abc'
s +='d'

In [ ]:
s

In [ ]:
id(s)

In [ ]:
s += 'dddd'
id(s)